In [1]:
downloaded = False

In [2]:
# !pip install fugashi[unidic-lite]
!pip install fugashi[unidic]
!pip install spacy
if not downloaded:
    !python -m unidic download
    !python -m spacy download ja_core_news_sm
    !python -m spacy download en_core_web_sm
    downloaded = True
import xml.etree.ElementTree as ET
import os
import time
import fugashi
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data import Field, BucketIterator, TabularDataset
import spacy
import random
import math
import gensim
from torchtext.data.metrics import bleu_score
from gensim.models import Word2Vec

     |████████████████████████████████| 486 kB 2.9 MB/s eta 0:00:01
  Created wheel for unidic: filename=unidic-1.0.3-py3-none-any.whl size=5498 sha256=0855fc73603b1378f327e6533b76e570c6830fe3f2277f18e50b09d870898a4c
  Stored in directory: /root/.cache/pip/wheels/23/30/0b/128289fb595ef4117d2976ffdbef5069ef83be813e88caa0a6
Successfully built unidic
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic.zip
Dictionary version: 2.3.0+2020-10-08
unidic.zip: 100%|████████████████████████████| 608M/608M [00:42<00:00, 14.3MB/s]
Finished download.
Downloaded UniDic v2.3.0+2020-10-08 to /opt/conda/lib/python3.7/site-packages/unidic/dicdir
     |████████████████████████████████| 7.6 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 2.6 MB/s eta 0:00:011


     |████████████████████████████████| 473 kB 9.0 MB/s eta 0:00:01
  Created wheel for ja-core-news-sm: filename=ja_core_news_sm-2.3.2-py3-none-any.whl size=7572681 sha256=5fe295e0d1b5e39e343eb79057ae2bb78c201014d5fd2c88f8522c6a707a9cfe
  Stored in directory: /tmp/pip-ephem-wheel-cache-wo5box7j/wheels/eb/24/fa/ca64ace11b329d1158b5a5300ff5f3a27ceebdb09852194bac
  Created wheel for sudachidict-core: filename=SudachiDict_core-20201223.post1-py3-none-any.whl size=71405681 sha256=65cb1ee647bdaa94daad5319b571e4182772e779f402fe73e2ca1072282255b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-wo5box7j/wheels/06/79/40/f404626b8327cceb1c1756e6b79b0dd9c3ad79c9d13aed7644
  Created wheel for sudachipy: filename=SudachiPy-0.5.1-cp37-cp37m-linux_x86_64.whl size=911872 sha256=906fbdd7e48634da0862082c445011694aa73b3d8077b302fb9d9cb261ef62a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-wo5box7j/wheels/6d/00/62/d1337cfacee73930ca5a3162508f03df43ba2110effa673f44
Successfully built ja-core-news-sm

# Wikipedia Corpus TSV Generation

In [3]:
# https://stackoverflow.com/questions/1912434/how-do-i-parse-xml-in-python
# https://docs.python.org/3/library/xml.etree.elementtree.html
# corpus = xml.etree.ElementTree.fromstring(all_of_it, parser=xml.etree.ElementTree.XMLParser(encoding='utf-8')).text
def createEnglishJapanesePairList(filepath, output) -> list:
    print(f'creating english japanese tsv (filepath:{filepath})')

    with open(output, mode="w", encoding="utf-8") as file_in:
        for path, subdirs, files in os.walk(filepath):
            try:
                for name in files:

                    corpus = ET.parse(os.path.join(path, name)).getroot()

                    #from the root, get all children that follow this node heirarchy, two different types of text blocks so two different for loops
                    # print(len(corpus.findall('par/sen')))
                    # print(len(corpus.findall('sec/par/sen')))

                    for type_tag in corpus.findall('par/sen'):
                        value = list(type_tag) # get all children elements under the <sen> tag
                        japanese_child = value[0].text # 1st child
                        english_child = value[5].text # 6th child

                        if english_child and japanese_child:
                            if ("(" in english_child and ")" in english_child) and not ("(" in japanese_child and ")" in japanese_child):
                                continue
                            file_in.write(english_child + "\t" + japanese_child + "\n")


                    for type_tag in corpus.findall('sec/par/sen'):
                        value = list(type_tag) # get all children elements under the <sen> tag
                        japanese_child = value[0].text # 1st child
                        english_child = value[5].text # 6th child

                        if english_child and japanese_child:
                            if ("(" in english_child and ")" in english_child) and not ("(" in japanese_child and ")" in japanese_child):
                                continue
                            file_in.write(english_child + "\t" + japanese_child + "\n")
            except:
                pass
    print('tsv saved!')

In [4]:
# create wikipedia tsv
# createEnglishJapanesePairList('../input/japaneseenglish-bilingual-corpus/wiki_corpus_2.01', 'wikipedia_raw.tsv')

# Combine Stanford Dataset

In [5]:
def combine_stanford_dataset(dataset_path, output):
    with open(output, mode="w", encoding="utf-8") as file_write:
        with open(dataset_path + "/train", mode="r", encoding="utf-8") as file_train:
            for index, line in enumerate(file_train):
                if index > 0:
                    file_write.writelines(line)
        with open(dataset_path + "/test", mode="r", encoding="utf-8") as file_test:
            for index, line in enumerate(file_test):
                if index > 0:
                    file_write.writelines(line)
        with open(dataset_path + "/dev", mode="r", encoding="utf-8") as file_dev:
            for index, line in enumerate(file_dev):
                if index > 0:
                    file_write.writelines(line)

In [6]:
# combine stanford dataset
# combine_stanford_dataset("../input/japanesse-english-subtittle-corpusjesc-cleaned", "stanford_raw.tsv")

# Generate Vectors for Embeddings

In [7]:
SOS_token = "<SOS>"
EOS_token = "<EOS>"
tagger = fugashi.Tagger()

# generates sentences for gensim's word2vec encoder
# uses fugashi to tokenize japanese text
class weebCorpus:
    filename = ""
    language = "japanese"
    
    def __init__(self, c, l):
        self.corpus = c
        self.language = l
    
    def __iter__(self):
        tokenized_text = list()
        for file in self.corpus:
            for line in open(file, mode="r", encoding="utf-8"):
                if self.language == "japanese":
                    text = line.split("\t")[1]
                    tokenized_text = [SOS_token] + [word.surface for word in tagger(text)] + [EOS_token]
                else: # english
                    text = line.split("\t")[0]
                    tokenized_text = [SOS_token] + nltk.word_tokenize(text)
                    tokenized_text.append(EOS_token)
                yield tokenized_text

def genVectors(corpus, language = "japanese"):
    sentences = weebCorpus(corpus, language)
    model = Word2Vec(sentences=sentences,
                     size=300,
                     window=5,
                     workers=2, # 2 if using GPU on Kaggle, 4+ if not
                     min_count=5,
                     iter=5)
        
    model.save("./" + language + "_vectors.model")
    # model.wv.save_word2vec_format("./" + language + "vectors_readable.txt", binary=False)

In [8]:
pretrained_dir = "../input/vectorized-jesc-wikipedia-raw"

if not os.path.exists(pretrained_dir):
    # start time to calculate time it takes to generate vectors
    start = time.time()
    files = ["stanford_raw.tsv", "wikipedia_raw.tsv"]

    genVectors(files, language="japanese")
    genVectors(files, language="english")

    print(time.time() - start)
    pretrained_dir = ""
    
 
print("most similar to 時代: ")
w1 = ["時代"]
japW2V = gensim.models.Word2Vec.load(os.path.join(pretrained_dir, "japanese_vectors.model"))
for thing in japW2V.wv.most_similar(positive=w1,topn=6):
    print (thing[0])

print("most similar to rice: ")
w1 = ["rice"]
engW2V = gensim.models.Word2Vec.load(os.path.join(pretrained_dir, "english_vectors.model"))
for thing in engW2V.wv.most_similar(positive=w1,topn=6):
    print (thing[0])

most similar to 時代: 
中期
後期
期
前期
近世
末期
most similar to rice: 
wheat
soup
sauce
vegetables
meat
mochi


# Initialize CUDA and load Generated Word Vectors

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer

In [10]:
def tokenizeJapanese(string):
    return [word.surface for word in tagger(string)]

# Encoder

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, embedding_size, hidden_size, num_layers, dropout): # input_size = number of tokens in Japanese, hidden_size=rand number
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(japW2V.wv.vectors))
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, (hidden_state, cell_state) = self.lstm(embedded)
        
        return hidden_state, cell_state

# Decoder

In [12]:
class DecoderRNN(nn.Module):
    def __init__(self, embedding_size, hidden_size, output_size, num_layers, dropout): # hidden_size=rand number (same that EncoderDNN() used, output_size = total tokens of English
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(engW2V.wv.vectors))
        self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden_state, cell_state):
        x = input.unsqueeze(0)
        embedding = self.dropout(self.embedding(x))
        outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)

        return predictions, hidden_state, cell_state

# Load Dataset

In [13]:
JAPANESE = Field(tokenize=tokenizeJapanese,
               lower=True,
               init_token=SOS_token,
               eos_token=EOS_token)

ENGLISH = Field(tokenize=nltk.tokenize.word_tokenize,
               lower=True,
               init_token=SOS_token,
               eos_token=EOS_token)

fields = [
  ('en', ENGLISH),
  ('ja', JAPANESE)
]

stanford_data = TabularDataset(
   path = os.path.join(pretrained_dir, 'stanford_raw.tsv'),
   format = 'tsv',
   fields = fields,
   skip_header = False
)

stanford_train, stanford_test = stanford_data.split(split_ratio=0.9)

/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/opt/conda/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/opt/conda/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warn

In [14]:
print(f"Number of training examples: {len(stanford_train.examples)}")
print(f"Number of test examples: {len(stanford_test.examples)}")

JAPANESE.build_vocab(stanford_train, min_freq=3)
ENGLISH.build_vocab(stanford_test, min_freq=3)

print(f"Unique tokens in source (japanese) vocabulary: {len(JAPANESE.vocab)}")
print(f"Unique tokens in target (english) vocabulary: {len(ENGLISH.vocab)}")

Number of training examples: 2138314
Number of test examples: 237591
Unique tokens in source (japanese) vocabulary: 60917
Unique tokens in target (english) vocabulary: 20189


In [27]:
train_iterator, valid_iterator = BucketIterator.splits(
    (stanford_train, stanford_test),
    batch_size = 128,
    sort_within_batch=True,
    sort_key=lambda x: len(x.ja),
    device = device
)

In [28]:
# Encoder/decoder params
embedding_size = 300
hidden_size = 1024
num_layers = 2
dropout = 0.5
output_size = len(ENGLISH.vocab)

In [29]:
encoder_lstm = EncoderRNN(embedding_size, hidden_size, num_layers, dropout).to(device)
decoder_lstm = DecoderRNN(embedding_size, hidden_size, output_size, num_layers, dropout).to(device)

# Encoder to Decoder Module

In [30]:
class Seq2Seq(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Seq2Seq, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder

    def forward(self, source, target, tfr=0.5):
        # Shape - Source : (10, 32) [(Sentence length German + some padding), Number of Sentences]
        batch_size = source.shape[1]

        # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
        target_len = target.shape[0]
        target_vocab_size = len(ENGLISH.vocab)

        # Shape --> outputs (14, 32, 5766)
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
        hidden_state, cell_state = self.Encoder(source)

        # Shape of x (32 elements)
        x = target[0]  # Trigger token <SOS>

        for i in range(1, target_len):
            # Shape --> output (32, 5766)
            output, hidden_state, cell_state = self.Decoder(x, hidden_state, cell_state)
            outputs[i] = output
            best_guess = output.argmax(1)  # 0th dimension is batch size, 1st dimension is word embedding
            x = target[i] if random.random() < tfr else best_guess  # Either pass the next word correctly from the dataset or use the earlier predicted word

        # Shape --> outputs (14, 32, 5766)
        return outputs

In [31]:
# initialize main model and optimizer
model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
def translate_sentence(model, sentence, japanese, english, device, max_length=50):
    tokens = [japanese.init_token]
    if type(sentence) == str:
        tokens.extend(tokenizeJapanese(sentence))
    else:
        tokens.extend([token.lower() for token in sentence])
    tokens.append(japanese.eos_token)
    text_to_indices = [japanese.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder(sentence_tensor)

    outputs = [english.vocab.stoi[SOS_token]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi[EOS_token]:
            break

        outputs.append(best_guess)

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

In [33]:
def bleu(data, model, japanese, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["ja"]
        trg = vars(example)["en"]
        
        prediction = translate_sentence(model, src, japanese, english, device)
        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

In [34]:
def checkpoint_and_save(model, best_loss, epoch, optimizer):
    print('saving')
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, 'checkpoint')
    torch.save(model.state_dict(),'checkpoint-sd')

In [35]:
pad_idx = ENGLISH.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [36]:
def train(model, optimizer, num_epochs, example_sentence):
    epoch_loss = 0.0
    best_loss = math.inf
    best_epoch = -math.inf
    for epoch in range(num_epochs):
        print("Epoch - {} / {}".format(epoch + 1, num_epochs))
        model.eval()
        translated_sentence = translate_sentence(model, example_sentence, JAPANESE, ENGLISH, device, 50)
        print(f"Translated example sentence: \n {translated_sentence}\n")

        model.train(True)
        for batch_idx, batch in enumerate(train_iterator):
            input = batch.ja.to(device)
            target = batch.en.to(device)

            # Pass the input and target for model's forward method
            output = model(input, target)
            output = output[1:].reshape(-1, output.shape[2])
            target = target[1:].reshape(-1)

            # Clear the accumulating gradients
            optimizer.zero_grad()

            # Calculate the loss value for every epoch
            loss = criterion(output, target)

            # Calculate the gradients for weights & biases using back-propagation
            loss.backward()

            # Clip the gradient value if it exceeds > 1
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

            # Update the weights values using the gradients we calculated using bp
            optimizer.step()
            epoch_loss += loss.item()

        if loss.item() < best_loss:
            best_loss = loss.item()
            best_epoch = epoch
            checkpoint_and_save(model, best_loss, epoch, optimizer) 
        elif (epoch - best_epoch) >= 10:
            print("no improvement in 10 epochs, break")
            break

        print("Epoch_Loss - {}".format(loss.item()))

    print(epoch_loss / len(train_iterator))

    score = bleu(test_ds[:100], model, JAPANESE, ENGLISH, device)
    print(f'BLEU score: {score*100:.2f}')

In [ ]:
sentence = "今お前などに興味はない。" # i am not interested in you now.
train(model, optimizer, 200, sentence)

Epoch - 1 / 200
Translated example sentence: 
 ['makeup', 'makeup', 'makeup', 'makeup', 'spacetime', 'spacetime', 'makeup', 'makeup', 'spacetime', 'makeup', 'spacetime', 'makeup', 'spacetime', 'makeup', 'makeup', 'spacetime', 'spacetime', 'makeup', 'makeup', 'spacetime', 'makeup', 'spacetime', 'makeup', 'spacetime', 'makeup', 'makeup', 'spacetime', 'spacetime', 'makeup', 'makeup', 'spacetime', 'makeup', 'spacetime', 'makeup', 'spacetime', 'makeup', 'makeup', 'spacetime', 'spacetime', 'makeup', 'makeup', 'spacetime', 'makeup', 'spacetime', 'makeup', 'spacetime', 'makeup', 'makeup', 'spacetime', 'spacetime']

